<a href="https://colab.research.google.com/github/mancap314/miscellanous/blob/master/cnn_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the companion notebook to this [blog article](https://macap314.github.io/transforming-keras-model-into-tensorflow-estimator.html), explaining how to transform a Keras model into a Tensorflow `Estimator`

# Creating a tf `Estimator` from a Keras model

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np



## Step 1: Importing data

In [2]:
# 1.0: Define some constants first
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
num_classes = 10

# 1.1: Import data
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

# 1.2: Format data
# convert class vectors to binary class matrices
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
eval_labels = tf.keras.utils.to_categorical(eval_labels, num_classes)

train_data = train_data / np.float32(255.0)
eval_data = eval_data /np.float32(255.0)

train_data = train_data.reshape(train_data.shape[0], img_rows, img_cols, 1)
eval_data = eval_data.reshape(eval_data.shape[0], img_rows, img_cols, 1)


11493376/11490434 [==============================] - 0s 0us/step


##  Step 2: Build model

In [3]:
# 2.1: Define model
model_cnn_0 = tf.keras.models.Sequential()
model_cnn_0.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,
                 name='x'
                ))
model_cnn_0.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn_0.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_cnn_0.add(tf.keras.layers.Dropout(0.25))
model_cnn_0.add(tf.keras.layers.Flatten())
model_cnn_0.add(tf.keras.layers.Dense(128, activation='relu'))
model_cnn_0.add(tf.keras.layers.Dropout(0.5))
model_cnn_0.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_cnn_0.summary()

# 2.2: Compile model
model_cnn_0.compile(loss=tf.keras.losses.categorical_crossentropy,
                    optimizer=tf.keras.optimizers.Adadelta(),
                    metrics=['accuracy'])


# 2.3: Transform the keras `model` into a tf `Estimator`
est_cnn_0 = tf.keras.estimator.model_to_estimator(keras_model=model_cnn_0)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (Conv2D)                   (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)        

## Step 3: Train Model

In [4]:
# 3.1: Define training input function
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x_input': train_data},  # 'x_input' because name of 1st layer is 'x', `model_cnn_0.input_names`
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# 3.2 Train estimator
est_cnn_0.train(input_fn=train_input_fn, steps=2000)


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpavjc6s9d/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpavjc6s9d/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: x/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: x/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchang

## Evaluate (trained) model

In [5]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x_input': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = est_cnn_0.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Starting evaluation at 2019-04-09T09:57:40Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpavjc6s9d/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-09-09:57:41
INFO:tensorflow:Saving dict for global step 2000: categorical_accuracy = 0.9884, global_step = 2000, loss = 0.034530666
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpavjc6s9d/model.ckpt-2000
{'categorical_accuracy': 0.9884, 'loss': 0.034530666, 'global_step': 2000}


# Create an `Estimator` from a pre-trained Keras model

## Step 1: Build, train and persist trained model

In [6]:
# 1.1: Define the model
model_cnn_0 = tf.keras.models.Sequential()
model_cnn_0.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,
                 name='x'
                ))
model_cnn_0.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn_0.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_cnn_0.add(tf.keras.layers.Dropout(0.25))
model_cnn_0.add(tf.keras.layers.Flatten())
model_cnn_0.add(tf.keras.layers.Dense(128, activation='relu'))
model_cnn_0.add(tf.keras.layers.Dropout(0.5))
model_cnn_0.add(tf.keras.layers.Dense(num_classes, activation='softmax'))



# 1.2: Compile model
model_cnn_0.compile(loss=tf.keras.losses.categorical_crossentropy,
                    optimizer=tf.keras.optimizers.Adadelta(),
                    metrics=['accuracy']
                   )

# 1.3: Create callbacks
# Checkpoint that will stop the training after 3 epochs without improvement of the accuracy on the validation data
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)
# Checkpoint that will persist the best model according to the accuracy on the validation data
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_acc')
callbacks=[early_stopping_callback, model_checkpoint_callback]

# 1.4: Train the model
model_cnn_0.fit(train_data, train_labels, validation_data=(eval_data, eval_labels), 
          epochs=2000, 
          verbose=0, 
          callbacks=callbacks)

## Step 2: transform the persisted trained model into an `Estimator`

In [7]:
est_cnn_0_trained = tf.keras.estimator.model_to_estimator(keras_model_path='best_model.h5')

INFO:tensorflow:Using default config.
INFO:tensorflow:Loading models from best_model.h5
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpliagcnq_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd69a63ceb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Step 3: Evaluate the resulting `Estimator`

In [8]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x_input': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = est_cnn_0_trained.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpliagcnq_, running initialization to evaluate.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpliagcnq_/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpliagcnq_/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: x/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: x/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: de

## Last step: Using the estimator to make predictions

In [11]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x_input': eval_data},
    y=None,
    num_epochs=1,
    shuffle=False)

predictions = est_cnn_0_trained.predict(input_fn=predict_input_fn)
predictions = np.array([list(p.values())[0] for p in list(predictions)])

print('predictions:\n\n{}'.format(predictions))

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpliagcnq_, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpliagcnq_/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpliagcnq_/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: x/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: x/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: den